In [15]:
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.layer = nn.Linear(7, 13)
        
    def forward(self, inputs):
        output = self.layer(inputs)
        output = F.relu(output)
        return output

In [11]:
lstm = LSTMModel()
n = 10
dim_in = 7
dim_out = 13

In [18]:
inputs = torch.randn(n, dim_in)
labels = torch.randn(n, dim_out)

torch.Size([10, 7]) torch.Size([10, 13])


In [19]:
outputs = lstm(inputs)